**The following code cells contains our experiments code details** 
---- 


In [ ]:
# algorithm 

!pip install learn2learn
!pip install flow-torch
import numpy as np
import matplotlib.pyplot as plt
import torch

from flow.flow import Sequential, inv_flow
from flow.conditioner import MADE
from flow.transformer import DSF
from flow.transformer import Affine as AffineTransformer
from flow.modules import BatchNorm, Affine, Sigmoid, Shuffle
from flow.training import get_device, train, plot_losses, test_nll

import learn2learn as l2l
from torch import nn, optim 

**EXP1** 
---
**separate flow for each class**   

**fully connected neural network for calssification**   

**5 ways 1 shots**

**10 iterations**

**batch size 8**

normalizing flow of 

BatchNorm Layer 

MADE Layer

----

```

for each task 
  for each class 
      train flow 
      generate data (15 images)
  fast adapt using generated data 

        
```
----
**Results**

**Time 38:01 min**

Mean accuracy/loss, stddev, and confidence intervals

(0.5367083439944933, 0.09340435423700708, 0.0074739049160712865)

In [ ]:
def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    # print("data:  ",data.shape)
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def fast_adapt_train (batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # print("data:  ",data.shape)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    # train on shots*ways*10 images => 5*1*7*=35 image 
    adaptation_indices[np.arange(shots*ways*7) * 2] = True
    
    evaluation_indices = torch.from_numpy(~adaptation_indices)

    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]
    # print(adaptation_data.shape)
    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def train_flow (trainX , valX , device):

  
  # trainX has a shape of (count , channel , pixles,pixles)
  

  trainX = torch.flatten(trainX,1,-1)
  valX = torch.flatten(valX , 1,-1)
  dim=trainX.shape[1]
  flow = Sequential(      

# very slow in sampling 
    # inv_flow(Sigmoid)(dim=dim),
    # BatchNorm(dim=dim),
    # MADE(DSF(dim=dim)),
    # Affine(dim=dim),  

      BatchNorm(dim=dim),
      MADE(AffineTransformer(dim=dim)),
  ).to(device)


  train_losses, val_losses = train(flow, trainX, valX, patience=10)
  # with torch.no_grad():
  #   sample = flow.sample(1000).cpu()
  #   # u = flow.forward(sample.to(device) ).cpu()

  # mean =torch.mean(sample , dim = 0 , keepdim=True) 
  # var = torch.var(sample , dim = 0 , keepdim=True)

  with torch.no_grad():
    train_data = flow.sample(15).cpu()

  # return train_data, train_losses , val_losses , mean , var
  return torch.reshape(train_data , (15,1,28,28))

In [ ]:

ways=5
shots=1
meta_lr=0.003
fast_lr=0.5
meta_batch_size=8
adaptation_steps=1
num_iterations=10
cuda=True
# seed=42
# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                                  train_ways=ways,
                                                  train_samples=2*shots,
                                                  test_ways=ways,
                                                  test_samples=2*shots,
                                                  num_tasks=20000,
                                                  root='~/data',
    )



/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "



Extracting /root/data/omniglot-py/images_background.zip to /root/data/omniglot-py



Extracting /root/data/omniglot-py/images_evaluation.zip to /root/data/omniglot-py


In [ ]:


device = torch.device('cpu')
if cuda:
  # torch.cuda.manual_seed(seed)
  device = torch.device('cuda')

# Create model
model = l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')


from tqdm import tqdm

for iteration in tqdm(range(num_iterations)):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0

    for task in range(meta_batch_size):
        # Compute meta-training loss
        # learner = maml.clone()

        # generate batch the batch contains # ways*shots , channels , img size  and labels 
        train_batch = tasksets.train.sample()
        val_batch = tasksets.validation.sample()

        # split batch based on classes in labels then   labels.split(int(data.shape[0]/ways))
        train_data, train_labels = train_batch
        val_data , val_labels= val_batch
        
        train_labels = torch.split( train_labels, int(train_data.shape[0]/ways))
        train_data = torch.split( train_data, int(train_data.shape[0]/ways))

        # val_data  = torch.split( val_data, int(train_data.shape[0]/ways))


        # python list that will contain val data and labels sorted 
        val_data_list = []
        val_label_list =[]
        train_data_generator = []
        train_lable_generator = []
        classes_mean ={}
        classes_var = {}
        
        for x , y in zip(train_data,train_labels):
          
          # now i have samples count , ch , img  [8, 1, 28, 28] for a single class 
          # train a flow 
          #  generate 1000 samples 
          # calculate the mean and variance 

          # as the validation data classes are not the same order as training data 
          # so we need to get validation data that hase the same class as the current training data 

          # current class indx 
          idx = ((val_labels == y[0]).nonzero(as_tuple=True)[0])
          val_data_list.append(val_data[idx])
          val_label_list.append(val_labels[idx])

          data  =   train_flow(x , val_data[idx] , get_device())
          
          # append the generated samples to new train data 
          train_data_generator.append(data)
          # print(data.shape)

          # we choose 15 here as the returned data size is 15 
          train_lable_generator.append( torch.ones((15))*y[0])
          
          # 'will be used to train a bayes classifier'
          # # append mean and var to classes mean and var dict 
          # classes_mean[y[0].numpy()]=mean
          # classes_var [y[0].numpy()]=var
        

        # don't know if will need them or no 
        # now both are sorted as the training data 
        val_data_sorted   = torch.stack(val_data_list)
        val_labels_sorted = torch.stack(val_data_list)


        train_data_generator = torch.cat(train_data_generator)
        train_lable_generator = torch.cat(train_lable_generator)
        
        batch = [train_data_generator ,train_lable_generator]

        learner = maml.clone()            
        
        evaluation_error, evaluation_accuracy = fast_adapt_train(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

# ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

        # Compute meta-validation loss
        batch = tasksets.validation.sample()
        
        learner = maml.clone()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,learner,loss,adaptation_steps,shots,ways,
                                                            device)
        meta_valid_error += evaluation_error.item()
        meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    print('Meta Valid Error', meta_valid_error / meta_batch_size)
    print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()



11it [00:01, 10.18it/s, epoch_progress=100%, train_loss=-7.331e+03, last_val_loss=+5.468e+20, best_epoch=1, best_loss=+7.393e+13]
11it [00:00, 12.95it/s, epoch_progress=100%, train_loss=-8.025e+03, last_val_loss=+1.142e+22, best_epoch=1, best_loss=+9.036e+14]
11it [00:00, 13.41it/s, epoch_progress=100%, train_loss=-7.350e+03, last_val_loss=+2.053e+22, best_epoch=1, best_loss=+7.934e+14]
11it [00:00, 13.66it/s, epoch_progress=100%, train_loss=-7.603e+03, last_val_loss=+1.040e+20, best_epoch=1, best_loss=+3.629e+13]
11it [00:00, 13.47it/s, epoch_progress=100%, train_loss=-7.124e+03, last_val_loss=+2.150e+20, best_epoch=1, best_loss=+1.449e+14]
14it [00:03,  4.31it/s, epoch_progress=100%, train_loss=-6.698e+03, last_val_loss=+4.086e+15, best_epoch=4, best_loss=+1.372e+11]
11it [00:00, 13.49it/s, epoch_progress=100%, train_loss=-7.140e+03, last_val_loss=+4.981e+18, best_epoch=1, best_loss=+7.726e+13]
11it [00:00, 13.56it/s, epoch_progress=100%, train_loss=-7.506e+03, last_val_loss=+6.997e+



Iteration 0
Meta Train Error 1.4690056592226028
Meta Train Accuracy 0.4468750059604645
Meta Valid Error 1.4572104662656784
Meta Valid Accuracy 0.4250000100582838


13it [00:10,  1.24it/s, epoch_progress=100%, train_loss=-6.822e+03, last_val_loss=+1.196e+16, best_epoch=3, best_loss=+2.664e+12]
11it [00:01,  6.36it/s, epoch_progress=100%, train_loss=-7.198e+03, last_val_loss=+2.463e+16, best_epoch=1, best_loss=+3.370e+11]
11it [00:00, 13.34it/s, epoch_progress=100%, train_loss=-7.197e+03, last_val_loss=+6.558e+21, best_epoch=1, best_loss=+6.660e+14]
11it [00:00, 13.05it/s, epoch_progress=100%, train_loss=-7.206e+03, last_val_loss=+3.017e+21, best_epoch=1, best_loss=+1.456e+13]
11it [00:02,  5.00it/s, epoch_progress=100%, train_loss=-7.662e+03, last_val_loss=+3.200e+17, best_epoch=1, best_loss=+8.938e+10]
11it [00:00, 13.02it/s, epoch_progress=100%, train_loss=-8.189e+03, last_val_loss=+8.517e+19, best_epoch=1, best_loss=+1.142e+14]
11it [00:00, 12.87it/s, epoch_progress=100%, train_loss=-7.705e+03, last_val_loss=+7.460e+20, best_epoch=1, best_loss=+1.071e+14]
11it [00:00, 12.62it/s, epoch_progress=100%, train_loss=-7.617e+03, last_val_loss=+3.454e+



Iteration 1
Meta Train Error 1.442554071545601
Meta Train Accuracy 0.37187500670552254
Meta Valid Error 1.5294938832521439
Meta Valid Accuracy 0.37500000558793545


11it [00:00, 12.94it/s, epoch_progress=100%, train_loss=-8.121e+03, last_val_loss=+8.529e+16, best_epoch=1, best_loss=+8.420e+12]
11it [00:00, 12.28it/s, epoch_progress=100%, train_loss=-7.527e+03, last_val_loss=+7.225e+21, best_epoch=1, best_loss=+1.665e+14]
11it [00:01,  7.79it/s, epoch_progress=100%, train_loss=-7.829e+03, last_val_loss=+3.997e+21, best_epoch=1, best_loss=+7.151e+14]
11it [00:00, 12.71it/s, epoch_progress=100%, train_loss=-7.174e+03, last_val_loss=+4.145e+20, best_epoch=1, best_loss=+9.769e+12]
11it [00:00, 13.06it/s, epoch_progress=100%, train_loss=-7.881e+03, last_val_loss=+3.245e+21, best_epoch=1, best_loss=+3.979e+14]
11it [00:00, 12.81it/s, epoch_progress=100%, train_loss=-7.385e+03, last_val_loss=+3.482e+19, best_epoch=1, best_loss=+1.805e+14]
11it [00:01,  6.81it/s, epoch_progress=100%, train_loss=-6.944e+03, last_val_loss=+8.056e+17, best_epoch=1, best_loss=+3.257e+13]
11it [00:00, 12.78it/s, epoch_progress=100%, train_loss=-7.803e+03, last_val_loss=+1.925e+

...



Iteration 7
Meta Train Error 1.1210977882146835
Meta Train Accuracy 0.6156250163912773
Meta Valid Error 1.1479889303445816
Meta Valid Accuracy 0.6000000145286322


11it [00:00, 12.32it/s, epoch_progress=100%, train_loss=-7.779e+03, last_val_loss=+6.969e+20, best_epoch=1, best_loss=+1.778e+13]
11it [00:00, 13.06it/s, epoch_progress=100%, train_loss=-7.061e+03, last_val_loss=+3.299e+18, best_epoch=1, best_loss=+1.049e+13]
11it [00:01,  6.31it/s, epoch_progress=100%, train_loss=-8.506e+03, last_val_loss=+1.781e+20, best_epoch=1, best_loss=+1.637e+13]
11it [00:04,  2.36it/s, epoch_progress=100%, train_loss=-7.943e+03, last_val_loss=+1.294e+20, best_epoch=1, best_loss=+2.176e+13]
11it [00:00, 12.74it/s, epoch_progress=100%, train_loss=-6.396e+03, last_val_loss=+6.842e+18, best_epoch=1, best_loss=+1.502e+12]
11it [00:00, 12.23it/s, epoch_progress=100%, train_loss=-7.012e+03, last_val_loss=+9.430e+18, best_epoch=1, best_loss=+3.525e+13]
11it [00:01,  8.47it/s, epoch_progress=100%, train_loss=-7.116e+03, last_val_loss=+5.026e+19, best_epoch=1, best_loss=+2.383e+13]
11it [00:01,  9.05it/s, epoch_progress=100%, train_loss=-7.862e+03, last_val_loss=+3.072e+



Iteration 8
Meta Train Error 1.1003836393356323
Meta Train Accuracy 0.6187500059604645
Meta Valid Error 1.5873451456427574
Meta Valid Accuracy 0.35000001080334187


11it [00:00, 12.88it/s, epoch_progress=100%, train_loss=-7.917e+03, last_val_loss=+1.084e+21, best_epoch=1, best_loss=+1.135e+14]
11it [00:00, 12.76it/s, epoch_progress=100%, train_loss=-7.487e+03, last_val_loss=+8.709e+16, best_epoch=1, best_loss=+7.852e+13]
11it [00:02,  4.76it/s, epoch_progress=100%, train_loss=-7.324e+03, last_val_loss=+2.818e+21, best_epoch=1, best_loss=+8.432e+13]
11it [00:00, 12.16it/s, epoch_progress=100%, train_loss=-7.360e+03, last_val_loss=+1.342e+17, best_epoch=1, best_loss=+3.133e+11]
11it [00:00, 13.00it/s, epoch_progress=100%, train_loss=-7.528e+03, last_val_loss=+4.495e+18, best_epoch=1, best_loss=+8.351e+10]
11it [00:00, 12.70it/s, epoch_progress=100%, train_loss=-7.248e+03, last_val_loss=+1.482e+16, best_epoch=1, best_loss=+1.136e+11]
11it [00:02,  4.89it/s, epoch_progress=100%, train_loss=-7.259e+03, last_val_loss=+3.837e+18, best_epoch=1, best_loss=+1.659e+14]
11it [00:00, 12.79it/s, epoch_progress=100%, train_loss=-7.158e+03, last_val_loss=+1.382e+



Iteration 9
Meta Train Error 1.0210638865828514
Meta Train Accuracy 0.6531250104308128
Meta Valid Error 1.5094431638717651
Meta Valid Accuracy 0.4000000096857548


CI 95%

In [ ]:
acc_acuracy=[]
iterations=600

for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

Mean accuracy/loss, stddev, and confidence intervals
(0.5367083439944933, 0.09340435423700708, 0.0074739049160712865)


In [ ]:
print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

0.27500000409781933
0.9000000059604645


**Random tasks accuracy**

In [ ]:

meta_test_error = 0.0
meta_test_accuracy = 0.0
for task in range(meta_batch_size):

    # Compute meta-testing loss
    learner = maml.clone()
    batch = tasksets.test.sample()
    evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                        learner,
                                                        loss,
                                                        3,
                                                        shots,
                                                        ways,
                                                        device)
    meta_test_error += evaluation_error.item()
    meta_test_accuracy += evaluation_accuracy.item()
print('Meta Test Error', meta_test_error / meta_batch_size)
print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)


Meta Test Error 0.896394494920969
Meta Test Accuracy 0.7250000070780516


In [ ]:

meta_test_error = 0.0
meta_test_accuracy = 0.0
for task in range(meta_batch_size):

    # Compute meta-testing loss
    learner = maml.clone()
    batch = tasksets.test.sample()
    evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                        learner,
                                                        loss,
                                                        10,
                                                        shots,
                                                        ways,
                                                        device)
    meta_test_error += evaluation_error.item()
    meta_test_accuracy += evaluation_accuracy.item()
print('Meta Test Error', meta_test_error / meta_batch_size)
print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)

Meta Test Error 1.317772735375911
Meta Test Accuracy 0.7000000104308128


**EXP 2**
---

**separate flow for each class**  
 
**Convolutional neural network for calssification**    

**5 ways 1 shots**

**10 iterations** 

**batch size 8**

normalizing flow of 

BatchNorm Layer 

MADE Layer

----

```

for each task 
  for each class 
      train flow 
      generate data (15 images)
  fast adapt using generated data 

        
```
----
**Results** 

**time 39:59 min**

Mean accuracy/loss, stddev, and confidence intervals

(0.5691250121283035, 0.08361534639018414, 0.006690621155185938)
   

In [ ]:
def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    # print("data:  ",data.shape)
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def fast_adapt_train (batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # print("data:  ",data.shape)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    # train on shots*ways*10 images => 5*1*7*=35 image 
    adaptation_indices[np.arange(shots*ways*7) * 2] = True
    
    evaluation_indices = torch.from_numpy(~adaptation_indices)

    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]
    # print(adaptation_data.shape)
    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def train_flow (trainX , valX , device):

  
  # trainX has a shape of (count , channel , pixles,pixles)
  

  trainX = torch.flatten(trainX,1,-1)
  valX = torch.flatten(valX , 1,-1)
  dim=trainX.shape[1]
  flow = Sequential(      

# very slow in sampling 
    # inv_flow(Sigmoid)(dim=dim),
    # BatchNorm(dim=dim),
    # MADE(DSF(dim=dim)),
    # Affine(dim=dim),  

      BatchNorm(dim=dim),
      MADE(AffineTransformer(dim=dim)),
  ).to(device)


  train_losses, val_losses = train(flow, trainX, valX, patience=10)
  # with torch.no_grad():
  #   sample = flow.sample(1000).cpu()
  #   # u = flow.forward(sample.to(device) ).cpu()

  # mean =torch.mean(sample , dim = 0 , keepdim=True) 
  # var = torch.var(sample , dim = 0 , keepdim=True)

  with torch.no_grad():
    train_data = flow.sample(15).cpu()

  # return train_data, train_losses , val_losses , mean , var
  return torch.reshape(train_data , (15,1,28,28))

In [ ]:
ways=5
shots=1
meta_lr=0.003
fast_lr=0.5
meta_batch_size=8
adaptation_steps=1
num_iterations=10
cuda=True
# seed=42
# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                                  train_ways=ways,
                                                  train_samples=2*shots,
                                                  test_ways=ways,
                                                  test_samples=2*shots,
                                                  num_tasks=20000,
                                                  root='~/data',
    )


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


In [ ]:

device = torch.device('cpu')
if cuda:
  # torch.cuda.manual_seed(seed)
  device = torch.device('cuda')

# Create model

model = l2l.vision.models.OmniglotCNN(ways)
# model = l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')


from tqdm import tqdm

for iteration in tqdm(range(num_iterations)):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0

    for task in range(meta_batch_size):
        # Compute meta-training loss
        # learner = maml.clone()

        # generate batch the batch contains # ways*shots , channels , img size  and labels 
        train_batch = tasksets.train.sample()
        val_batch = tasksets.validation.sample()

        # split batch based on classes in labels then   labels.split(int(data.shape[0]/ways))
        train_data, train_labels = train_batch
        val_data , val_labels= val_batch
        
        train_labels = torch.split( train_labels, int(train_data.shape[0]/ways))
        train_data = torch.split( train_data, int(train_data.shape[0]/ways))

        # val_data  = torch.split( val_data, int(train_data.shape[0]/ways))


        # python list that will contain val data and labels sorted 
        val_data_list = []
        val_label_list =[]
        train_data_generator = []
        train_lable_generator = []
        classes_mean ={}
        classes_var = {}
        
        for x , y in zip(train_data,train_labels):
          
          # now i have samples count , ch , img  [8, 1, 28, 28] for a single class 
          # train a flow 
          #  generate 1000 samples 
          # calculate the mean and variance 

          # as the validation data classes are not the same order as training data 
          # so we need to get validation data that hase the same class as the current training data 

          # current class indx 
          idx = ((val_labels == y[0]).nonzero(as_tuple=True)[0])
          val_data_list.append(val_data[idx])
          val_label_list.append(val_labels[idx])

          data  =   train_flow(x , val_data[idx] , get_device())
          
          # append the generated samples to new train data 
          train_data_generator.append(data)
          # print(data.shape)

          # we choose 15 here as the returned data size is 15 
          train_lable_generator.append( torch.ones((15))*y[0])
          
          # 'will be used to train a bayes classifier'
          # # append mean and var to classes mean and var dict 
          # classes_mean[y[0].numpy()]=mean
          # classes_var [y[0].numpy()]=var
        

        # don't know if will need them or no 
        # now both are sorted as the training data 
        val_data_sorted   = torch.stack(val_data_list)
        val_labels_sorted = torch.stack(val_data_list)


        train_data_generator = torch.cat(train_data_generator)
        train_lable_generator = torch.cat(train_lable_generator)
        
        batch = [train_data_generator ,train_lable_generator]

        learner = maml.clone()            
        
        evaluation_error, evaluation_accuracy = fast_adapt_train(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

# ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

        # Compute meta-validation loss
        batch = tasksets.validation.sample()
        
        learner = maml.clone()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,learner,loss,adaptation_steps,shots,ways,
                                                            device)
        meta_valid_error += evaluation_error.item()
        meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    print('Meta Valid Error', meta_valid_error / meta_batch_size)
    print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()

11it [00:00, 11.83it/s, epoch_progress=100%, train_loss=-7.699e+03, last_val_loss=+2.381e+18, best_epoch=1, best_loss=+1.715e+11]
11it [00:00, 11.77it/s, epoch_progress=100%, train_loss=-6.649e+03, last_val_loss=+8.628e+17, best_epoch=1, best_loss=+2.016e+12]
11it [00:00, 12.04it/s, epoch_progress=100%, train_loss=-7.615e+03, last_val_loss=+9.128e+16, best_epoch=1, best_loss=+1.590e+12]
11it [00:00, 12.40it/s, epoch_progress=100%, train_loss=-6.809e+03, last_val_loss=+1.405e+21, best_epoch=1, best_loss=+9.153e+13]
11it [00:00, 11.83it/s, epoch_progress=100%, train_loss=-7.865e+03, last_val_loss=+8.399e+20, best_epoch=1, best_loss=+1.986e+13]
11it [00:00, 11.41it/s, epoch_progress=100%, train_loss=-7.995e+03, last_val_loss=+3.081e+20, best_epoch=1, best_loss=+3.168e+14]
11it [00:00, 11.44it/s, epoch_progress=100%, train_loss=-7.273e+03, last_val_loss=+6.064e+17, best_epoch=1, best_loss=+3.989e+12]
11it [00:00, 11.10it/s, epoch_progress=100%, train_loss=-7.499e+03, last_val_loss=+1.307e+



Iteration 0
Meta Train Error 1.8304346650838852
Meta Train Accuracy 0.496875012293458
Meta Valid Error 2.520754411816597
Meta Valid Accuracy 0.3750000074505806


11it [00:00, 12.66it/s, epoch_progress=100%, train_loss=-7.865e+03, last_val_loss=+1.841e+19, best_epoch=1, best_loss=+4.702e+12]
11it [00:00, 12.37it/s, epoch_progress=100%, train_loss=-7.861e+03, last_val_loss=+4.841e+20, best_epoch=1, best_loss=+6.887e+13]
11it [00:00, 12.04it/s, epoch_progress=100%, train_loss=-7.466e+03, last_val_loss=+5.032e+19, best_epoch=1, best_loss=+5.602e+12]
11it [00:01,  5.65it/s, epoch_progress=100%, train_loss=-7.962e+03, last_val_loss=+5.557e+18, best_epoch=1, best_loss=+1.295e+14]
11it [00:00, 12.05it/s, epoch_progress=100%, train_loss=-7.639e+03, last_val_loss=+7.450e+20, best_epoch=1, best_loss=+1.769e+13]
11it [00:00, 12.13it/s, epoch_progress=100%, train_loss=-7.569e+03, last_val_loss=+1.742e+21, best_epoch=1, best_loss=+5.022e+13]
11it [00:00, 12.61it/s, epoch_progress=100%, train_loss=-7.815e+03, last_val_loss=+7.079e+19, best_epoch=1, best_loss=+1.501e+14]
11it [00:02,  4.24it/s, epoch_progress=100%, train_loss=-6.970e+03, last_val_loss=+2.426e+



Iteration 1
Meta Train Error 1.46099753677845
Meta Train Accuracy 0.5468750074505806
Meta Valid Error 2.518529698252678
Meta Valid Accuracy 0.3000000063329935


11it [00:00, 12.57it/s, epoch_progress=100%, train_loss=-7.691e+03, last_val_loss=+4.256e+19, best_epoch=1, best_loss=+2.121e+13]
11it [00:02,  5.00it/s, epoch_progress=100%, train_loss=-6.573e+03, last_val_loss=+5.535e+19, best_epoch=1, best_loss=+7.396e+12]
11it [00:00, 11.79it/s, epoch_progress=100%, train_loss=-8.033e+03, last_val_loss=+7.635e+20, best_epoch=1, best_loss=+3.414e+13]
11it [00:00, 11.86it/s, epoch_progress=100%, train_loss=-7.212e+03, last_val_loss=+1.221e+14, best_epoch=1, best_loss=+3.820e+09]
11it [00:00, 11.74it/s, epoch_progress=100%, train_loss=-6.950e+03, last_val_loss=+3.172e+19, best_epoch=1, best_loss=+1.264e+14]
11it [00:00, 11.90it/s, epoch_progress=100%, train_loss=-8.131e+03, last_val_loss=+1.377e+22, best_epoch=1, best_loss=+4.835e+14]
11it [00:00, 12.29it/s, epoch_progress=100%, train_loss=-8.008e+03, last_val_loss=+7.405e+20, best_epoch=1, best_loss=+4.679e+13]
11it [00:02,  4.89it/s, epoch_progress=100%, train_loss=-7.384e+03, last_val_loss=+4.620e+

...



Iteration 7
Meta Train Error 0.4199875919148326
Meta Train Accuracy 0.8718750178813934
Meta Valid Error 2.041491612792015
Meta Valid Accuracy 0.3750000074505806


12it [00:04,  2.71it/s, epoch_progress=100%, train_loss=-8.549e+03, last_val_loss=+5.182e+16, best_epoch=2, best_loss=+1.467e+12]
11it [00:02,  5.06it/s, epoch_progress=100%, train_loss=-6.573e+03, last_val_loss=+1.243e+19, best_epoch=1, best_loss=+2.358e+13]
11it [00:00, 11.63it/s, epoch_progress=100%, train_loss=-7.528e+03, last_val_loss=+1.504e+21, best_epoch=1, best_loss=+2.509e+14]
11it [00:00, 12.04it/s, epoch_progress=100%, train_loss=-7.276e+03, last_val_loss=+1.672e+20, best_epoch=1, best_loss=+1.191e+14]
11it [00:00, 11.65it/s, epoch_progress=100%, train_loss=-7.357e+03, last_val_loss=+9.561e+21, best_epoch=1, best_loss=+2.368e+14]
11it [00:01,  9.91it/s, epoch_progress=100%, train_loss=-8.135e+03, last_val_loss=+6.437e+20, best_epoch=1, best_loss=+1.732e+13]
11it [00:00, 11.62it/s, epoch_progress=100%, train_loss=-7.247e+03, last_val_loss=+1.147e+20, best_epoch=1, best_loss=+1.015e+14]
11it [00:02,  4.39it/s, epoch_progress=100%, train_loss=-7.480e+03, last_val_loss=+1.243e+



Iteration 8
Meta Train Error 0.625767694786191
Meta Train Accuracy 0.7531250044703484
Meta Valid Error 1.5699550807476044
Meta Valid Accuracy 0.475000012665987


11it [00:00, 11.26it/s, epoch_progress=100%, train_loss=-7.380e+03, last_val_loss=+3.904e+19, best_epoch=1, best_loss=+5.189e+13]
11it [00:01,  6.30it/s, epoch_progress=100%, train_loss=-6.803e+03, last_val_loss=+3.901e+18, best_epoch=1, best_loss=+9.309e+13]
11it [00:02,  5.23it/s, epoch_progress=100%, train_loss=-7.495e+03, last_val_loss=+3.810e+19, best_epoch=1, best_loss=+3.414e+12]
13it [00:03,  4.25it/s, epoch_progress=100%, train_loss=-7.426e+03, last_val_loss=+1.001e+14, best_epoch=3, best_loss=+2.565e+12]
11it [00:00, 12.12it/s, epoch_progress=100%, train_loss=-6.937e+03, last_val_loss=+1.365e+22, best_epoch=1, best_loss=+1.092e+14]
11it [00:00, 12.60it/s, epoch_progress=100%, train_loss=-7.084e+03, last_val_loss=+8.883e+20, best_epoch=1, best_loss=+1.007e+14]
11it [00:00, 12.52it/s, epoch_progress=100%, train_loss=-7.686e+03, last_val_loss=+3.953e+20, best_epoch=1, best_loss=+4.976e+12]
11it [00:01,  6.14it/s, epoch_progress=100%, train_loss=-7.519e+03, last_val_loss=+1.915e+



Iteration 9
Meta Train Error 0.5504261367022991
Meta Train Accuracy 0.8093750029802322
Meta Valid Error 2.0175972804427147
Meta Valid Accuracy 0.35000000707805157


CI 95%

In [ ]:
acc_acuracy=[]
iterations=600

for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

Mean accuracy/loss, stddev, and confidence intervals
(0.5691250121283035, 0.08361534639018414, 0.006690621155185938)


In [ ]:
print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

0.3250000048428774
0.8250000104308128


**Random Task accuracy**

In [ ]:
meta_test_error = 0.0
meta_test_accuracy = 0.0
for task in range(meta_batch_size):

    # Compute meta-testing loss
    learner = maml.clone()
    batch = tasksets.test.sample()
    evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                        learner,
                                                        loss,
                                                        5,
                                                        shots,
                                                        ways,
                                                        device)
    meta_test_error += evaluation_error.item()
    meta_test_accuracy += evaluation_accuracy.item()
print('Meta Test Error', meta_test_error / meta_batch_size)
print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)

Meta Test Error 0.8597511872649193
Meta Test Accuracy 0.7000000141561031


In [ ]:
meta_test_error = 0.0
meta_test_accuracy = 0.0
for task in range(meta_batch_size):

    # Compute meta-testing loss
    learner = maml.clone()
    batch = tasksets.test.sample()
    evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                        learner,
                                                        loss,
                                                        3,
                                                        shots,
                                                        ways,
                                                        device)
    meta_test_error += evaluation_error.item()
    meta_test_accuracy += evaluation_accuracy.item()
print('Meta Test Error', meta_test_error / meta_batch_size)
print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)

Meta Test Error 0.7645831368863583
Meta Test Accuracy 0.725000012665987


**EXP3** 
---
**train the flow conditioned on the classes** 

**fully connected neural network for calssification**   

**5 ways 1 shots**

**10 iterations** 

**batch size 8**

normalizing flow of 

BatchNorm Layer 

MADE Layer

----

```

for each task 
    train flow conditioned on task's classes
    generate data (75 images)
  fast adapt using generated data 

        
```
----
**Results** 

**time 08:04 min**

Mean accuracy/loss, stddev, and confidence intervals

(0.5365833435983708, 0.09937928944376485, 0.007951999304494365)



In [ ]:
def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    # print("data:  ",data.shape)
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def fast_adapt_train(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    # print("labels:  ",labels.shape)
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(37) * 2] = True

    evaluation_indices = torch.from_numpy(~adaptation_indices)

    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]
    # print(adaptation_data.shape)
    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy

def train_flow (trainX ,cond_train, valX ,cond_val, device):

  
  # trainX has a shape of (count , channel , pixles,pixles)
  

  trainX = torch.flatten(trainX,1,-1)
  valX = torch.flatten(valX , 1,-1)
  dim=trainX.shape[1]
  
  # print(cond_train)
  cond_train = torch.nn.functional.one_hot(cond_train)
  cond_val   = torch.nn.functional.one_hot(cond_val) 
  # print(cond_train)
  cond_dim = cond_train.size(1)
  flow = Sequential(      

# very slow in sampling 
    # inv_flow(Sigmoid)(dim=dim),
    # BatchNorm(dim=dim),
    # MADE(DSF(dim=dim)),
    # Affine(dim=dim),  

      BatchNorm(dim=dim),
      MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),
  ).to(device)


  train_losses, val_losses = train(flow, trainX, valX ,cond_train=cond_train, cond_val=cond_val , patience=10 )
 
  test_labels = np.random.choice(5, size=(75))
  test_labels = torch.from_numpy(test_labels)
  test_cond = torch.nn.functional.one_hot(test_labels)

  with torch.no_grad():
    train_data = flow.sample(75,cond=test_cond.to(device)).cpu()

  # return train_data, train_losses , val_losses , mean , var
  return torch.reshape(train_data , (75,1,28,28)) , test_labels




In [ ]:
ways=5
shots=1
meta_lr=0.003
fast_lr=0.5
meta_batch_size=8
adaptation_steps=1
num_iterations=10
cuda=True
# seed=42
# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                                  train_ways=ways,
                                                  train_samples=2*shots,
                                                  test_ways=ways,
                                                  test_samples=2*shots,
                                                  num_tasks=20000,
                                                  root='~/data',
    )



/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


In [ ]:

device = torch.device('cpu')
if cuda:
  # torch.cuda.manual_seed(seed)
  device = torch.device('cuda')

# Create model
model = l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')


from tqdm import tqdm

for iteration in tqdm(range(num_iterations)):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0

    for task in range(meta_batch_size):
        # Compute meta-training loss
        # learner = maml.clone()

        # generate batch the batch contains # ways*shots , channels , img size  and labels 
        train_batch = tasksets.train.sample()
        val_batch = tasksets.validation.sample()

        # split batch based on classes in labels then   labels.split(int(data.shape[0]/ways))
        train_data, train_labels = train_batch
        val_data , val_labels= val_batch

        
        train_data_generator ,train_lable_generator  = train_flow (train_data ,train_labels , val_data, val_labels , device)

        # train_data_generator = torch.cat(train_data_generator)
        # train_lable_generator = torch.cat(train_lable_generator)
        
        batch = [train_data_generator ,train_lable_generator]

        learner = maml.clone()            
        
        evaluation_error, evaluation_accuracy = fast_adapt_train(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

# ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

        # Compute meta-validation loss
        batch = tasksets.validation.sample()
        
        learner = maml.clone()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,learner,loss,adaptation_steps,shots,ways,
                                                            device)
        meta_valid_error += evaluation_error.item()
        meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    print('Meta Valid Error', meta_valid_error / meta_batch_size)
    print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()


11it [00:01, 10.91it/s, epoch_progress=100%, train_loss=-5.463e+03, last_val_loss=+1.272e+17, best_epoch=1, best_loss=+1.010e+13]
11it [00:00, 11.65it/s, epoch_progress=100%, train_loss=-5.328e+03, last_val_loss=+5.343e+19, best_epoch=1, best_loss=+5.334e+12]
11it [00:00, 11.94it/s, epoch_progress=100%, train_loss=-5.233e+03, last_val_loss=+2.409e+19, best_epoch=1, best_loss=+4.610e+12]
11it [00:00, 11.85it/s, epoch_progress=100%, train_loss=-4.585e+03, last_val_loss=+4.572e+13, best_epoch=1, best_loss=+3.721e+09]
11it [00:00, 11.74it/s, epoch_progress=100%, train_loss=-4.100e+03, last_val_loss=+1.525e+17, best_epoch=1, best_loss=+1.749e+12]
11it [00:00, 11.76it/s, epoch_progress=100%, train_loss=-4.313e+03, last_val_loss=+6.099e+14, best_epoch=1, best_loss=+1.150e+10]
11it [00:00, 11.20it/s, epoch_progress=100%, train_loss=-4.370e+03, last_val_loss=+1.761e+17, best_epoch=1, best_loss=+2.662e+11]
11it [00:00, 11.44it/s, epoch_progress=100%, train_loss=-5.280e+03, last_val_loss=+2.483e+



Iteration 0
Meta Train Error 1.640379637479782
Meta Train Accuracy 0.20723684318363667
Meta Valid Error 1.5225267857313156
Meta Valid Accuracy 0.3500000052154064


11it [00:00, 11.49it/s, epoch_progress=100%, train_loss=-3.585e+03, last_val_loss=+1.510e+15, best_epoch=1, best_loss=+1.453e+10]
11it [00:00, 11.35it/s, epoch_progress=100%, train_loss=-5.097e+03, last_val_loss=+1.331e+19, best_epoch=1, best_loss=+2.868e+13]
11it [00:00, 11.88it/s, epoch_progress=100%, train_loss=-3.561e+03, last_val_loss=+1.554e+18, best_epoch=1, best_loss=+6.142e+11]
11it [00:00, 11.93it/s, epoch_progress=100%, train_loss=-5.030e+03, last_val_loss=+4.666e+18, best_epoch=1, best_loss=+1.633e+11]
11it [00:00, 11.95it/s, epoch_progress=100%, train_loss=-4.384e+03, last_val_loss=+1.562e+19, best_epoch=1, best_loss=+3.163e+11]
11it [00:00, 11.69it/s, epoch_progress=100%, train_loss=-4.546e+03, last_val_loss=+5.021e+20, best_epoch=1, best_loss=+1.444e+13]
11it [00:00, 12.10it/s, epoch_progress=100%, train_loss=-4.193e+03, last_val_loss=+9.559e+18, best_epoch=1, best_loss=+3.990e+12]
11it [00:00, 11.74it/s, epoch_progress=100%, train_loss=-5.076e+03, last_val_loss=+7.975e+



Iteration 1
Meta Train Error 1.6397636234760284
Meta Train Accuracy 0.20723683759570122
Meta Valid Error 1.4876786470413208
Meta Valid Accuracy 0.3500000089406967


12it [00:01,  7.44it/s, epoch_progress=100%, train_loss=-4.305e+03, last_val_loss=+6.655e+13, best_epoch=2, best_loss=+1.120e+10]
11it [00:00, 11.60it/s, epoch_progress=100%, train_loss=-4.610e+03, last_val_loss=+8.492e+17, best_epoch=1, best_loss=+5.669e+11]
11it [00:00, 11.52it/s, epoch_progress=100%, train_loss=-4.379e+03, last_val_loss=+2.820e+17, best_epoch=1, best_loss=+2.455e+12]
11it [00:00, 11.67it/s, epoch_progress=100%, train_loss=-5.265e+03, last_val_loss=+4.506e+17, best_epoch=1, best_loss=+3.317e+12]
11it [00:00, 11.35it/s, epoch_progress=100%, train_loss=-4.591e+03, last_val_loss=+1.492e+16, best_epoch=1, best_loss=+1.353e+10]
11it [00:00, 11.72it/s, epoch_progress=100%, train_loss=-6.189e+03, last_val_loss=+1.630e+18, best_epoch=1, best_loss=+1.709e+13]
11it [00:00, 11.71it/s, epoch_progress=100%, train_loss=-5.673e+03, last_val_loss=+1.271e+20, best_epoch=1, best_loss=+4.847e+13]
11it [00:00, 11.63it/s, epoch_progress=100%, train_loss=-4.968e+03, last_val_loss=+1.083e+

...



Iteration 7
Meta Train Error 1.6730516105890274
Meta Train Accuracy 0.1842105258256197
Meta Valid Error 1.6758798509836197
Meta Valid Accuracy 0.17500000447034836


11it [00:00, 11.92it/s, epoch_progress=100%, train_loss=-5.514e+03, last_val_loss=+2.095e+18, best_epoch=1, best_loss=+2.455e+12]
11it [00:00, 11.65it/s, epoch_progress=100%, train_loss=-3.686e+03, last_val_loss=+1.685e+19, best_epoch=1, best_loss=+7.797e+11]
11it [00:00, 11.57it/s, epoch_progress=100%, train_loss=-4.741e+03, last_val_loss=+4.193e+17, best_epoch=1, best_loss=+3.106e+12]
11it [00:00, 11.21it/s, epoch_progress=100%, train_loss=-4.152e+03, last_val_loss=+8.628e+17, best_epoch=1, best_loss=+4.289e+10]
11it [00:00, 11.28it/s, epoch_progress=100%, train_loss=-3.969e+03, last_val_loss=+1.898e+18, best_epoch=1, best_loss=+3.837e+11]
11it [00:00, 11.62it/s, epoch_progress=100%, train_loss=-3.477e+03, last_val_loss=+1.367e+17, best_epoch=1, best_loss=+1.749e+11]
12it [00:01,  8.07it/s, epoch_progress=100%, train_loss=-4.277e+03, last_val_loss=+1.541e+17, best_epoch=2, best_loss=+1.609e+12]
11it [00:00, 12.06it/s, epoch_progress=100%, train_loss=-5.042e+03, last_val_loss=+2.942e+



Iteration 8
Meta Train Error 1.6478504985570908
Meta Train Accuracy 0.18750000186264515
Meta Valid Error 1.6863006204366684
Meta Valid Accuracy 0.25000000558793545


11it [00:00, 12.06it/s, epoch_progress=100%, train_loss=-4.030e+03, last_val_loss=+5.039e+15, best_epoch=1, best_loss=+1.847e+09]
11it [00:00, 12.01it/s, epoch_progress=100%, train_loss=-3.451e+03, last_val_loss=+1.036e+17, best_epoch=1, best_loss=+1.138e+12]
11it [00:00, 11.78it/s, epoch_progress=100%, train_loss=-4.673e+03, last_val_loss=+2.050e+19, best_epoch=1, best_loss=+1.725e+12]
11it [00:00, 11.96it/s, epoch_progress=100%, train_loss=-4.347e+03, last_val_loss=+1.439e+20, best_epoch=1, best_loss=+5.935e+12]
11it [00:00, 11.51it/s, epoch_progress=100%, train_loss=-4.239e+03, last_val_loss=+4.014e+15, best_epoch=1, best_loss=+3.388e+12]
11it [00:00, 11.56it/s, epoch_progress=100%, train_loss=-4.773e+03, last_val_loss=+9.769e+18, best_epoch=1, best_loss=+2.612e+12]
11it [00:00, 11.12it/s, epoch_progress=100%, train_loss=-5.729e+03, last_val_loss=+4.475e+20, best_epoch=1, best_loss=+5.595e+13]
11it [00:00, 11.64it/s, epoch_progress=100%, train_loss=-4.535e+03, last_val_loss=+1.510e+



Iteration 9
Meta Train Error 1.6649517565965652
Meta Train Accuracy 0.19078947603702545
Meta Valid Error 1.4921004176139832
Meta Valid Accuracy 0.3500000089406967


CI 95%


In [ ]:
acc_acuracy=[]
iterations=600

for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

Mean accuracy/loss, stddev, and confidence intervals
(0.5365833435983708, 0.09937928944376485, 0.007951999304494365)


In [ ]:
print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

0.27500000409781933
0.800000011920929


**Random task accuracy**

In [ ]:
meta_test_error = 0.0
meta_test_accuracy = 0.0
for task in range(meta_batch_size):

    # Compute meta-testing loss
    learner = maml.clone()
    batch = tasksets.test.sample()
    evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                        learner,
                                                        loss,
                                                        3,
                                                        shots,
                                                        ways,
                                                        device)
    meta_test_error += evaluation_error.item()
    meta_test_accuracy += evaluation_accuracy.item()
print('Meta Test Error', meta_test_error / meta_batch_size)
print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)

Meta Test Error 1.940928265452385
Meta Test Accuracy 0.3000000063329935


In [ ]:
meta_test_error = 0.0
meta_test_accuracy = 0.0
for task in range(meta_batch_size):

    # Compute meta-testing loss
    learner = maml.clone()
    batch = tasksets.test.sample()
    evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                        learner,
                                                        loss,
                                                        3,
                                                        shots,
                                                        ways,
                                                        device)
    meta_test_error += evaluation_error.item()
    meta_test_accuracy += evaluation_accuracy.item()
print('Meta Test Error', meta_test_error / meta_batch_size)
print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)

Meta Test Error 0.6755867125466466
Meta Test Accuracy 0.8000000044703484


In [ ]:
meta_test_error = 0.0
meta_test_accuracy = 0.0
for task in range(meta_batch_size):

    # Compute meta-testing loss

    learner = maml.clone()
    batch = tasksets.test.sample()
    evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                        learner,
                                                        loss,
                                                       5,
                                                        shots,
                                                        ways,
                                                        device)
    meta_test_error += evaluation_error.item()
    meta_test_accuracy += evaluation_accuracy.item()
print('Meta Test Error', meta_test_error / meta_batch_size)
print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)

Meta Test Error 1.1156090898439288
Meta Test Accuracy 0.7000000085681677


**EXP 4**
---

**train the flow conditioned on the classes** 

**Convolutional neural network for calssification**   

**5 ways 1 shots**

**10 iterations** 

**batch size 8**

normalizing flow of 

BatchNorm Layer 

MADE Layer

----

```

for each task 
    train flow conditioned on task's classes
    generate data (75 images)
  fast adapt using generated data 

        
```
----
**Results** 

**time 08:08 min**

Mean accuracy/loss, stddev, and confidence intervals

(0.5772083453182131, 0.08377996003121009, 0.006703792989742969)



In [ ]:
def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    # print("data:  ",data.shape)
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def fast_adapt_train(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    # print("labels:  ",labels.shape)
    data, labels = data.to(device), labels.to(device, dtype=torch.int64)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(37) * 2] = True

    evaluation_indices = torch.from_numpy(~adaptation_indices)

    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]
    # print(adaptation_data.shape)
    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)


    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy

def train_flow (trainX ,cond_train, valX ,cond_val, device):

  
  # trainX has a shape of (count , channel , pixles,pixles)
  

  trainX = torch.flatten(trainX,1,-1)
  valX = torch.flatten(valX , 1,-1)
  dim=trainX.shape[1]
  
  # print(cond_train)
  cond_train = torch.nn.functional.one_hot(cond_train)
  cond_val   = torch.nn.functional.one_hot(cond_val) 
  # print(cond_train)
  cond_dim = cond_train.size(1)
  flow = Sequential(      

# very slow in sampling 
    # inv_flow(Sigmoid)(dim=dim),
    # BatchNorm(dim=dim),
    # MADE(DSF(dim=dim)),
    # Affine(dim=dim),  

      BatchNorm(dim=dim),
      MADE(AffineTransformer(dim=dim) , cond_dim=cond_dim),
  ).to(device)


  train_losses, val_losses = train(flow, trainX, valX ,cond_train=cond_train, cond_val=cond_val , patience=10 )
 
  test_labels = np.random.choice(5, size=(75))
  test_labels = torch.from_numpy(test_labels)
  test_cond = torch.nn.functional.one_hot(test_labels)

  with torch.no_grad():
    train_data = flow.sample(75,cond=test_cond.to(device)).cpu()

  # return train_data, train_losses , val_losses , mean , var
  return torch.reshape(train_data , (75,1,28,28)) , test_labels




In [ ]:
ways=5
shots=1
meta_lr=0.003
fast_lr=0.5
meta_batch_size=8
adaptation_steps=1
num_iterations=10
cuda=True
# seed=42
# Load train/validation/test tasksets using the benchmark interface
tasksets = l2l.vision.benchmarks.get_tasksets('omniglot',
                                                  train_ways=ways,
                                                  train_samples=2*shots,
                                                  test_ways=ways,
                                                  test_samples=2*shots,
                                                  num_tasks=20000,
                                                  root='~/data',
    )



/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
device = torch.device('cpu')
if cuda:
  # torch.cuda.manual_seed(seed)
  device = torch.device('cuda')

# Create model
model = l2l.vision.models.OmniglotCNN(ways)
# model = l2l.vision.models.OmniglotFC(28 ** 2, ways)
model.to(device)
maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')


from tqdm import tqdm

for iteration in tqdm(range(num_iterations)):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0

    for task in range(meta_batch_size):
        # Compute meta-training loss
        # learner = maml.clone()

        # generate batch the batch contains # ways*shots , channels , img size  and labels 
        train_batch = tasksets.train.sample()
        val_batch = tasksets.validation.sample()

        # split batch based on classes in labels then   labels.split(int(data.shape[0]/ways))
        train_data, train_labels = train_batch
        val_data , val_labels= val_batch

        
        train_data_generator ,train_lable_generator  = train_flow (train_data ,train_labels , val_data, val_labels , device)

        # train_data_generator = torch.cat(train_data_generator)
        # train_lable_generator = torch.cat(train_lable_generator)
        
        batch = [train_data_generator ,train_lable_generator]

        learner = maml.clone()            
        
        evaluation_error, evaluation_accuracy = fast_adapt_train(batch,
                                                            learner,
                                                            loss,
                                                            adaptation_steps,
                                                            shots,
                                                            ways,
                                                            device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

# ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

        # Compute meta-validation loss
        batch = tasksets.validation.sample()
        
        learner = maml.clone()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,learner,loss,adaptation_steps,shots,ways,
                                                            device)
        meta_valid_error += evaluation_error.item()
        meta_valid_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    print('Meta Valid Error', meta_valid_error / meta_batch_size)
    print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()


11it [00:01, 10.97it/s, epoch_progress=100%, train_loss=-5.792e+03, last_val_loss=+1.122e+18, best_epoch=1, best_loss=+3.547e+11]
11it [00:00, 11.48it/s, epoch_progress=100%, train_loss=-4.691e+03, last_val_loss=+6.838e+11, best_epoch=1, best_loss=+7.315e+07]
11it [00:00, 11.73it/s, epoch_progress=100%, train_loss=-4.255e+03, last_val_loss=+7.798e+13, best_epoch=1, best_loss=+1.338e+10]
12it [00:01,  7.85it/s, epoch_progress=100%, train_loss=-4.640e+03, last_val_loss=+6.173e+16, best_epoch=2, best_loss=+2.833e+13]
11it [00:00, 11.80it/s, epoch_progress=100%, train_loss=-4.928e+03, last_val_loss=+1.025e+17, best_epoch=1, best_loss=+1.166e+11]
11it [00:00, 12.13it/s, epoch_progress=100%, train_loss=-4.745e+03, last_val_loss=+2.632e+18, best_epoch=1, best_loss=+7.486e+10]
11it [00:00, 11.93it/s, epoch_progress=100%, train_loss=-4.085e+03, last_val_loss=+2.717e+17, best_epoch=1, best_loss=+2.874e+10]
11it [00:00, 12.08it/s, epoch_progress=100%, train_loss=-5.003e+03, last_val_loss=+6.313e+



Iteration 0
Meta Train Error 2.7678613662719727
Meta Train Accuracy 0.24342105351388454
Meta Valid Error 2.1582921892404556
Meta Valid Accuracy 0.4250000137835741


11it [00:00, 12.09it/s, epoch_progress=100%, train_loss=-4.661e+03, last_val_loss=+1.018e+15, best_epoch=1, best_loss=+2.851e+09]
11it [00:00, 12.02it/s, epoch_progress=100%, train_loss=-4.709e+03, last_val_loss=+4.352e+18, best_epoch=1, best_loss=+6.695e+12]
11it [00:00, 11.54it/s, epoch_progress=100%, train_loss=-4.849e+03, last_val_loss=+5.677e+16, best_epoch=1, best_loss=+9.780e+11]
11it [00:00, 11.42it/s, epoch_progress=100%, train_loss=-5.448e+03, last_val_loss=+3.524e+17, best_epoch=1, best_loss=+1.298e+13]
11it [00:00, 11.70it/s, epoch_progress=100%, train_loss=-4.934e+03, last_val_loss=+1.467e+19, best_epoch=1, best_loss=+8.220e+11]
11it [00:00, 11.56it/s, epoch_progress=100%, train_loss=-4.747e+03, last_val_loss=+1.300e+18, best_epoch=1, best_loss=+2.006e+12]
11it [00:00, 12.05it/s, epoch_progress=100%, train_loss=-5.490e+03, last_val_loss=+2.138e+19, best_epoch=1, best_loss=+9.755e+12]
12it [00:01,  7.43it/s, epoch_progress=100%, train_loss=-4.659e+03, last_val_loss=+5.954e+



Iteration 1
Meta Train Error 2.843963325023651
Meta Train Accuracy 0.19407894648611546
Meta Valid Error 2.506077229976654
Meta Valid Accuracy 0.30000000447034836


11it [00:00, 11.89it/s, epoch_progress=100%, train_loss=-4.115e+03, last_val_loss=+1.379e+19, best_epoch=1, best_loss=+3.372e+11]
11it [00:00, 11.91it/s, epoch_progress=100%, train_loss=-5.259e+03, last_val_loss=+4.146e+19, best_epoch=1, best_loss=+1.257e+12]
11it [00:00, 11.52it/s, epoch_progress=100%, train_loss=-3.597e+03, last_val_loss=+5.914e+16, best_epoch=1, best_loss=+4.777e+09]
11it [00:00, 11.58it/s, epoch_progress=100%, train_loss=-4.333e+03, last_val_loss=+1.487e+17, best_epoch=1, best_loss=+3.193e+10]
11it [00:00, 11.92it/s, epoch_progress=100%, train_loss=-4.323e+03, last_val_loss=+1.172e+19, best_epoch=1, best_loss=+9.790e+11]
11it [00:00, 11.63it/s, epoch_progress=100%, train_loss=-5.500e+03, last_val_loss=+5.529e+20, best_epoch=1, best_loss=+1.939e+13]
11it [00:00, 11.74it/s, epoch_progress=100%, train_loss=-5.441e+03, last_val_loss=+3.030e+18, best_epoch=1, best_loss=+1.508e+12]
12it [00:01,  7.97it/s, epoch_progress=100%, train_loss=-4.202e+03, last_val_loss=+5.243e+

...



Iteration 7
Meta Train Error 2.564416065812111
Meta Train Accuracy 0.2335526291280985
Meta Valid Error 1.8769275546073914
Meta Valid Accuracy 0.500000013038516


11it [00:00, 11.89it/s, epoch_progress=100%, train_loss=-4.784e+03, last_val_loss=+1.376e+19, best_epoch=1, best_loss=+4.831e+11]
12it [00:01,  7.47it/s, epoch_progress=100%, train_loss=-4.824e+03, last_val_loss=+2.398e+16, best_epoch=2, best_loss=+3.564e+12]
11it [00:00, 11.97it/s, epoch_progress=100%, train_loss=-5.001e+03, last_val_loss=+1.299e+18, best_epoch=1, best_loss=+6.717e+12]
11it [00:00, 12.04it/s, epoch_progress=100%, train_loss=-4.447e+03, last_val_loss=+4.501e+19, best_epoch=1, best_loss=+4.287e+12]
11it [00:00, 12.00it/s, epoch_progress=100%, train_loss=-4.703e+03, last_val_loss=+1.044e+19, best_epoch=1, best_loss=+4.059e+14]
11it [00:00, 11.85it/s, epoch_progress=100%, train_loss=-5.663e+03, last_val_loss=+5.852e+19, best_epoch=1, best_loss=+1.078e+14]
11it [00:00, 11.46it/s, epoch_progress=100%, train_loss=-5.809e+03, last_val_loss=+1.119e+19, best_epoch=1, best_loss=+3.205e+12]
11it [00:00, 11.93it/s, epoch_progress=100%, train_loss=-3.985e+03, last_val_loss=+1.466e+



Iteration 8
Meta Train Error 2.4534398913383484
Meta Train Accuracy 0.22368421033024788
Meta Valid Error 1.7653998658061028
Meta Valid Accuracy 0.4500000160187483


11it [00:00, 12.08it/s, epoch_progress=100%, train_loss=-5.357e+03, last_val_loss=+3.492e+16, best_epoch=1, best_loss=+3.435e+12]
11it [00:00, 11.70it/s, epoch_progress=100%, train_loss=-5.084e+03, last_val_loss=+2.583e+17, best_epoch=1, best_loss=+7.178e+10]
11it [00:00, 11.78it/s, epoch_progress=100%, train_loss=-4.460e+03, last_val_loss=+6.766e+16, best_epoch=1, best_loss=+1.486e+11]
12it [00:01,  8.19it/s, epoch_progress=100%, train_loss=-3.932e+03, last_val_loss=+1.084e+16, best_epoch=2, best_loss=+2.144e+11]
11it [00:00, 11.64it/s, epoch_progress=100%, train_loss=-4.921e+03, last_val_loss=+3.727e+17, best_epoch=1, best_loss=+5.269e+11]
12it [00:01,  7.68it/s, epoch_progress=100%, train_loss=-3.763e+03, last_val_loss=+4.478e+16, best_epoch=2, best_loss=+1.109e+12]
11it [00:00, 11.84it/s, epoch_progress=100%, train_loss=-3.610e+03, last_val_loss=+2.567e+14, best_epoch=1, best_loss=+9.350e+08]
11it [00:00, 12.06it/s, epoch_progress=100%, train_loss=-5.463e+03, last_val_loss=+4.320e+



Iteration 9
Meta Train Error 2.5408490896224976
Meta Train Accuracy 0.22697368822991848
Meta Valid Error 1.9616957008838654
Meta Valid Accuracy 0.30000000819563866


CI 95%

In [ ]:
acc_acuracy=[]
iterations= 600
for _ in range(iterations):
  meta_test_error = 0.0
  meta_test_accuracy = 0.0

  for task in range(meta_batch_size):

      # Compute meta-testing loss
      learner = maml.clone()
      batch = tasksets.test.sample()
      evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                          learner,
                                                          loss,
                                                          3,
                                                          shots,
                                                          ways,
                                                          device)
      meta_test_error += evaluation_error.item()
      meta_test_accuracy += evaluation_accuracy.item()

  # print('Meta Test Error', meta_test_error / meta_batch_size)
  # print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)
  acc_acuracy.append(meta_test_accuracy/meta_batch_size)


acc_acuracy = np.array(acc_acuracy)
means = np.mean(acc_acuracy, 0)
stds = np.std(acc_acuracy, 0)
ci95 = 1.96*stds/np.sqrt(iterations)

print('Mean accuracy/loss, stddev, and confidence intervals')
print((means, stds, ci95))

Mean accuracy/loss, stddev, and confidence intervals
(0.5772083453182131, 0.08377996003121009, 0.006703792989742969)


In [ ]:
print(acc_acuracy[acc_acuracy.argmin()])
print(acc_acuracy[acc_acuracy.argmax()])

0.32500000670552254
0.800000011920929


**Result random task**

In [ ]:
meta_test_error = 0.0
meta_test_accuracy = 0.0
for task in range(meta_batch_size):

    # Compute meta-testing loss
    learner = maml.clone()
    batch = tasksets.test.sample()
    evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                        learner,
                                                        loss,
                                                        5,
                                                        shots,
                                                        ways,
                                                        device)
    meta_test_error += evaluation_error.item()
    meta_test_accuracy += evaluation_accuracy.item()
print('Meta Test Error', meta_test_error / meta_batch_size)
print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)

Meta Test Error 1.2538875718601048
Meta Test Accuracy 0.5687500131316483


In [ ]:
meta_test_error = 0.0
meta_test_accuracy = 0.0
for task in range(meta_batch_size):

    # Compute meta-testing loss
    learner = maml.clone()
    batch = tasksets.test.sample()
    evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                        learner,
                                                        loss,
                                                        5,
                                                        shots,
                                                        ways,
                                                        device)
    meta_test_error += evaluation_error.item()
    meta_test_accuracy += evaluation_accuracy.item()
print('Meta Test Error', meta_test_error / meta_batch_size)
print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)

Meta Test Error 1.1046872679144144
Meta Test Accuracy 0.6000000089406967


In [ ]:
meta_test_error = 0.0
meta_test_accuracy = 0.0
for task in range(meta_batch_size):

    # Compute meta-testing loss
    learner = maml.clone()
    batch = tasksets.test.sample()
    evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                        learner,
                                                        loss,
                                                        5,
                                                        shots,
                                                        ways,
                                                        device)
    meta_test_error += evaluation_error.item()
    meta_test_accuracy += evaluation_accuracy.item()
print('Meta Test Error', meta_test_error / meta_batch_size)
print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)

Meta Test Error 0.7923296503722668
Meta Test Accuracy 0.7500000111758709


In [ ]:
meta_test_error = 0.0
meta_test_accuracy = 0.0
for task in range(meta_batch_size):

    # Compute meta-testing loss
    learner = maml.clone()
    batch = tasksets.test.sample()
    evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                        learner,
                                                        loss,
                                                        3,
                                                        shots,
                                                        ways,
                                                        device)
    meta_test_error += evaluation_error.item()
    meta_test_accuracy += evaluation_accuracy.item()
print('Meta Test Error', meta_test_error / meta_batch_size)
print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)

Meta Test Error 0.8685418516397476
Meta Test Accuracy 0.625000013038516
